In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error, mean_absolute_error
from datetime import datetime
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

folder_path2 = "C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_2"


In [4]:
# train_test = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1/train_test.csv", index_col=False)

## Irrigation yearly

In [3]:
# Load the dataset
combined_irrigation = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1/combined_irrigation.csv", index_col=False)

# Clean the 'Qté eau (m3)' column
combined_irrigation['Qté eau (m3)'] = combined_irrigation['Qté eau (m3)'].str.replace(',', '.')
combined_irrigation['Qté eau (m3)'] = combined_irrigation['Qté eau (m3)'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

# Convert the 'Date' column to datetime format and extract the year
combined_irrigation['Date'] = pd.to_datetime(combined_irrigation['Date'])

C:\Users\DataScience\AppData\Local\Temp\ipykernel_81360\607629391.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_irrigation = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1/combined_irrigation.csv", index_col=False)
C:\Users\DataScience\AppData\Local\Temp\ipykernel_81360\607629391.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_irrigation['Date'] = pd.to_datetime(combined_irrigation['Date'])


In [4]:
combined_irrigation['Year'] = combined_irrigation['Date'].dt.year

# Calculate the yearly average of 'Qté eau (m3)' for each combination of the other columns
grouped_irrigation = combined_irrigation.groupby(['Year', 'Campagne', 'Region', 'Site', 'Famille', 'Variete', 'Num Parcelle', 'Code Traçabilité'])['Qté eau (m3)'].sum().reset_index()

# Rename the 'Qté eau (m3)' column to 'yearly_average'
grouped_irrigation.rename(columns={'Qté eau (m3)': 'yearly_sum_of_qte_d_eau'}, inplace=True)

# Now grouped_irrigation has the structure you want
grouped_irrigation

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,Code Traçabilité,yearly_sum_of_qte_d_eau
0,2020,20/21,GHARB,Ben Mansour,AF,AF1,11240,TR11240,3341.7000
1,2020,20/21,GHARB,Ben Mansour,CLA,CLA14,11211,TR11211,858.2684
2,2020,20/21,GHARB,Ben Mansour,CLA,CLA16,11160,TR11160,936.0316
3,2020,20/21,GHARB,Ben Mansour,ORA,ORA2,11170,TR11170,2142.0000
4,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA1,13030,TR11130,302.9532
...,...,...,...,...,...,...,...,...,...
4063,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10406,TR10600,8138.4881
4064,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10407,TR10500,8528.9743
4065,2023,22/23,TADLA,Bni Ayat,ORA,ORA9,20106,TR20400,6196.2969
4066,2023,22/23,TADLA,Bni Ayat,SPA,SPA1,20104,TR20104,3848.6437


In [8]:
grouped_irrigation[grouped_irrigation['yearly_sum_of_qte_d_eau'] == 0]

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,Code Traçabilité,yearly_sum_of_qte_d_eau


In [9]:
# Ensure the folder exists
if not os.path.exists(folder_path2):
    os.makedirs(folder_path2)

# Specify the full file path (folder path + file name)
file_path = os.path.join(folder_path2, 'irrigation_yearly.csv')

# Save the DataFrame to a CSV file
grouped_irrigation.to_csv(file_path, index=False)

## NPK yearly

In [5]:
combined_Apports_NPK = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1\combined_Apports_NPK.csv", index_col=False)

# Clean the 'Qté eau (m3)' column
combined_Apports_NPK['N'] = combined_Apports_NPK['N'].str.replace(',', '.')
combined_Apports_NPK['N'] = combined_Apports_NPK['N'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

combined_Apports_NPK['P'] = combined_Apports_NPK['P'].str.replace(',', '.')
combined_Apports_NPK['P'] = combined_Apports_NPK['P'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

combined_Apports_NPK['K'] = combined_Apports_NPK['K'].str.replace(',', '.')
combined_Apports_NPK['K'] = combined_Apports_NPK['K'].str.replace(r'[^\d.-]', '', regex=True).astype(float)

# Convert the 'Date' column to datetime format and extract the year
combined_Apports_NPK['Date'] = pd.to_datetime(combined_Apports_NPK['Date'])

C:\Users\DataScience\AppData\Local\Temp\ipykernel_81360\3309568967.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_Apports_NPK = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1\combined_Apports_NPK.csv", index_col=False)
C:\Users\DataScience\AppData\Local\Temp\ipykernel_81360\3309568967.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  combined_Apports_NPK['Date'] = pd.to_datetime(combined_Apports_NPK['Date'])


In [7]:
combined_Apports_NPK['Year'] = combined_Apports_NPK['Date'].dt.year

# Calculate the yearly average of 'N', 'P', and 'K' for each combination of the other columns
grouped_Apports_NPK = combined_Apports_NPK.groupby(['Year', 'Campagne', 'Region', 'Site', 'Famille', 'Variete', 'Num Parcelle', 'CodeTracabilite'])[['N', 'P', 'K']].sum().reset_index()

# Rename the columns to indicate they are yearly averages
grouped_Apports_NPK.rename(columns={
    'N': 'yearly_sum_N',
    'P': 'yearly_sum_P',
    'K': 'yearly_sum_K'}, inplace=True)

# View the first few rows of the resulting DataFrame
grouped_Apports_NPK

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,yearly_sum_N,yearly_sum_P,yearly_sum_K
0,2020,20/21,GHARB,Ben Mansour,AF,AF1,11240,TR11240,3.900000,0.000000,13.800000
1,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA1,13030,TR11130,20.347619,4.645285,0.000000
2,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA12,13010,TR11120,29.654580,6.770029,0.000000
3,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA14,12010,TR11020,9.232091,0.000000,0.023421
4,2020,20/21,GHARB,Chorf Laghouazi,CLA,CLA14,12010,TR11090,90.554194,30.871385,0.348981
...,...,...,...,...,...,...,...,...,...,...,...
3579,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10406.0,TR10600,115.533677,45.351586,70.971979
3580,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10407.0,TR10500,122.574110,38.571023,68.406656
3581,2023,22/23,TADLA,Bni Ayat,ORA,ORA9,20106.0,TR20400,77.300449,27.846275,138.532847
3582,2023,22/23,TADLA,Bni Ayat,SPA,SPA1,20104.0,TR20104,106.194203,36.373904,115.000710


In [8]:
grouped_Apports_NPK.isna().sum()  ##none null value

Year               0
Campagne           0
Region             0
Site               0
Famille            0
Variete            0
Num Parcelle       0
CodeTracabilite    0
yearly_sum_N       0
yearly_sum_P       0
yearly_sum_K       0
dtype: int64

In [ ]:
grouped_Apports_NPK

In [9]:
# Ensure the folder exists
if not os.path.exists(folder_path2):
    os.makedirs(folder_path2)

# Specify the full file path (folder path + file name)
file_path = os.path.join(folder_path2, 'Apports_NPK_yearly.csv')

# Save the DataFrame to a CSV file
grouped_Apports_NPK.to_csv(file_path, index=False)

## Test train yearly

In [10]:
train_test = pd.read_csv("C:/Users/DataScience/Desktop/kaggle's competition/AgrumeProdPred/processing_1/train_test.csv", index_col=False)

In [11]:
train_test = train_test.drop(columns='Id')
train_test["Date"] = pd.to_datetime(train_test["Date"])
train_test['Tonnage'] = train_test['Tonnage'].str.replace(',', '.').astype(float)

In [13]:
train_test['Year'] = train_test['Date'].dt.year

# Calculate the yearly average of 'Qté eau (m3)' for each combination of the other columns
grouped_train_test = train_test.groupby(['Year', 'Campagne', 'Region', 'Site', 'Famille', 'Variete', 'Num Parcelle', 'CodeTracabilite', 'is_train'])['Tonnage'].sum().reset_index()

# Rename the 'Qté eau (m3)' column to 'yearly_average'
grouped_train_test.rename(columns={'Tonnage': 'yearly_sum_tonnage'}, inplace=True)

# Now grouped_irrigation has the structure you want
grouped_train_test

,Year,Campagne,Region,Site,Famille,Variete,Num Parcelle,CodeTracabilite,is_train,yearly_sum_tonnage
0,2020,20/21,GHARB,Ben Mansour,AF,AF1,11240,TR11240,True,161.00000
1,2020,20/21,GHARB,Ben Mansour,CLA,CLA13,11130,TR11130,True,36082.22357
2,2020,20/21,GHARB,Ben Mansour,CLA,CLA13,11212,TR11212,True,48051.99275
3,2020,20/21,GHARB,Ben Mansour,CLA,CLA14,11211,TR11211,True,64649.20368
4,2020,20/21,GHARB,Ben Mansour,CLA,CLA15,11140,TR11140,True,46440.53500
...,...,...,...,...,...,...,...,...,...,...
2273,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10403,TR10500,False,0.00000
2274,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10406,TR10600,False,0.00000
2275,2023,22/23,TADLA,Bni Ayat,ORA,ORA6,10407,TR10500,False,0.00000
2276,2023,22/23,TADLA,Bni Ayat,ORA,ORA9,20106,TR20400,False,0.00000


In [14]:
grouped_train_test.isnull().sum()  ##none null value

Year                  0
Campagne              0
Region                0
Site                  0
Famille               0
Variete               0
Num Parcelle          0
CodeTracabilite       0
is_train              0
yearly_sum_tonnage    0
dtype: int64

In [15]:
# Ensure the folder exists
if not os.path.exists(folder_path2):
    os.makedirs(folder_path2)

# Specify the full file path (folder path + file name)
file_path = os.path.join(folder_path2, 'train_test_yearly.csv')

# Save the DataFrame to a CSV file
grouped_train_test.to_csv(file_path, index=False)